In [1]:
from Modified_lux3_wrapper.modified_wrappers_20250228_01 import ModifiedLuxAIS3GymEnv
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import SubprocVecEnv
import torch
import torch.nn as nn
import torch.nn.functional as F
# from torch.optim import AdamW
import os
# import copy
# from GreedyLRScheduler import GreedyLR
# from luxai_s3.wrappers import LuxAIS3GymEnv
import gc
gc.enable()
# from stable_baselines3.common.buffers import DictRolloutBuffer
# from tqdm.notebook import tqdm
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.policies import MultiInputActorCriticPolicy
from gymnasium import spaces

In [2]:
torch.set_float32_matmul_precision('high')
torch._dynamo.config.capture_scalar_outputs = True
torch._dynamo.config.cache_size_limit = 128
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
np.set_printoptions(linewidth=200)
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True,max_split_size_mb:512"
torch.backends.cudnn.benchmark = True
torch.cuda.set_per_process_memory_fraction(0.8)
torch.cuda.empty_cache()

In [3]:
learning_rate = 1e-6
env = ModifiedLuxAIS3GymEnv(numpy_output=True)
model = PPO("MultiInputPolicy", env, verbose=1, learning_rate=learning_rate, ent_coef=0.015, vf_coef=0.75, clip_range_vf=0.15, clip_range=0.2, n_steps=505, batch_size=101, max_grad_norm=0.5)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
common policies actorcriticpolicy _build_mlp_extractor


In [4]:
def normalize_observation(obs: dict, obs_space: spaces.Dict) -> dict:
    """
    Normalize continuous features in the observation dict using min-max scaling,
    while leaving discrete or binary features unchanged.
    """
    norm_obs = {}
    for key, space in obs_space.spaces.items():
        value = obs[key]
        # For Box spaces with numeric types (and not MultiBinary)
        if isinstance(space, spaces.Box) and np.issubdtype(space.dtype, np.number):
            # If the range is [0,1] (or binary), assume it's already normalized
            if (space.low == 0).all() and (space.high == 1).all():
                norm_obs[key] = value
            else:
                # Convert to float and apply min-max normalization:
                # norm = (value - low) / (high - low)
                low = torch.tensor(space.low, device=value.device, dtype=torch.float32)
                high = torch.tensor(space.high, device=value.device, dtype=torch.float32)
                # print(value)
                norm_obs[key] = (value.to(dtype=torch.float32) - low) / (high - low + 1e-8)
        else:
            # For discrete or MultiBinary spaces, just copy the values
            norm_obs[key] = value
    return norm_obs

In [5]:
class CustomFeatureExtractor(BaseFeaturesExtractor):
    """
    Custom feature extractor that:
    - Processes 24x24 grid features using CNN.
    - Flattens and concatenates other features.
    """

    def __init__(self, observation_space: spaces.Dict, features_dim: int = 0):
        super(CustomFeatureExtractor, self).__init__(observation_space, features_dim)

        # Identify 24x24 grid features
        self.grid_features = ["map_explored_status", "map_features_energy", "map_features_tile_type", "sensor_mask"]
        self.features = []
        for key in observation_space.keys():
            self.features.append(key)

        # **CNN for 24x24 Grid Features** (Expects input shape [batch, channels, 24, 24])
        self.cnn_extractor = nn.Sequential(
            nn.Conv2d(len(self.grid_features), 16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.SiLU(),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.SiLU(),
            nn.Flatten(),
            nn.Dropout(0.1),
        )

        self.cnn_extractor = torch.compile(self.cnn_extractor)

        # Compute CNN output dimension (using a dummy input)
        dummy_input = torch.zeros((1, len(self.grid_features), 24, 24))
        cnn_output_dim = self.cnn_extractor(dummy_input).shape[1]

        # **Flatten layers for non-grid features**
        self.extractors = nn.ModuleDict()
        flatten_dim = 0

        for key in observation_space.keys():
            # print(key)
            space_shape = observation_space.spaces[key].shape
            self.extractors[key] = nn.Flatten()
            flatten_dim += torch.prod(torch.tensor(space_shape)).item()

        # Compute total feature dimension
        self._features_dim = cnn_output_dim + flatten_dim

    def forward(self, observations):
        """
        Forward pass:
        - Grid features go through CNN
        - Other features are flattened
        - Both are concatenated into a single tensor
        """
        # observations = normalize_observation(observations, model.observation_space)

        grid_stack = torch.stack([observations[key] for key in self.grid_features], dim=1).float()
        grid_features = self.cnn_extractor(grid_stack)

        # Flatten vector features
        features = torch.cat([self.extractors[key](observations[key]) for key in self.features], dim=1)

        combined_features = torch.cat([grid_features, features], dim=1)

        return combined_features

In [6]:
policy_kwargs = dict(
    features_extractor_class=CustomFeatureExtractor,
    features_extractor_kwargs=dict(features_dim=20897),
    activation_fn=nn.SiLU,
    # net_arch=dict(pi=[8192, 4096, 2048, 1024], vf=[8192, 4096, 2048, 1024, 512, 256, 128, 64]),
    net_arch=dict(pi=[4096, 2048, 1024], vf=[4096, 2048, 1024, 512, 256, 128]),
    # net_arch=dict(pi=[128, 64], vf=[128, 64]),
)
learning_rate = 2e-5
env = ModifiedLuxAIS3GymEnv(numpy_output=True)
model = PPO(
    "MultiInputPolicy", env, policy_kwargs=policy_kwargs, verbose=0, learning_rate=learning_rate, ent_coef=0.015, vf_coef=0.75, clip_range_vf=0.3, clip_range=0.2, n_steps=505, batch_size=505,
    max_grad_norm=0.5, n_epochs=10, save_dir="saved_policies/", tensorboard_log="logs/", gamma=0.99, target_kl=None, gae_lambda=0.9
)

common policies actorcriticpolicy _build_mlp_extractor


In [7]:
model.policy

MultiInputActorCriticPolicy(
  (features_extractor): CustomFeatureExtractor(
    (cnn_extractor): OptimizedModule(
      (_orig_mod): Sequential(
        (0): Conv2d(4, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU()
        (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): SiLU()
        (6): Flatten(start_dim=1, end_dim=-1)
        (7): Dropout(p=0.1, inplace=False)
      )
    )
    (extractors): ModuleDict(
      (enemy_energies): Flatten(start_dim=1, end_dim=-1)
      (enemy_positions): Flatten(start_dim=1, end_dim=-1)
      (enemy_spawn_location): Flatten(start_dim=1, end_dim=-1)
      (enemy_visible_mask): Flatten(start_dim=1, end_dim=-1)
      (map_explored_status): Flatten(start_dim=1, end_dim=-1)
      (map_features_energy): Flatten(

In [8]:
model.policy_class.__init__.__annotations__

{'observation_space': gymnasium.spaces.dict.Dict,
 'action_space': gymnasium.spaces.space.Space,
 'lr_schedule': typing.Callable[[float], float],
 'net_arch': typing.Union[list[int], dict[str, list[int]], NoneType],
 'activation_fn': type[torch.nn.modules.module.Module],
 'ortho_init': bool,
 'use_sde': bool,
 'log_std_init': float,
 'full_std': bool,
 'use_expln': bool,
 'squash_output': bool,
 'features_extractor_class': type[stable_baselines3.common.torch_layers.BaseFeaturesExtractor],
 'features_extractor_kwargs': typing.Optional[dict[str, typing.Any]],
 'share_features_extractor': bool,
 'normalize_images': bool,
 'optimizer_class': type[torch.optim.optimizer.Optimizer],
 'optimizer_kwargs': typing.Optional[dict[str, typing.Any]]}

In [9]:
import copy
model.policy_2 = copy.deepcopy(model.policy)
model.policy_2

MultiInputActorCriticPolicy(
  (features_extractor): CustomFeatureExtractor(
    (cnn_extractor): OptimizedModule(
      (_orig_mod): Sequential(
        (0): Conv2d(4, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU()
        (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): SiLU()
        (6): Flatten(start_dim=1, end_dim=-1)
        (7): Dropout(p=0.1, inplace=False)
      )
    )
    (extractors): ModuleDict(
      (enemy_energies): Flatten(start_dim=1, end_dim=-1)
      (enemy_positions): Flatten(start_dim=1, end_dim=-1)
      (enemy_spawn_location): Flatten(start_dim=1, end_dim=-1)
      (enemy_visible_mask): Flatten(start_dim=1, end_dim=-1)
      (map_explored_status): Flatten(start_dim=1, end_dim=-1)
      (map_features_energy): Flatten(

In [10]:
model.policy.load_state_dict(torch.load("saved_policies_20250307_04/policy_120.pth"))
model.policy_2.load_state_dict(torch.load("saved_policies_20250307_04/policy_2_120.pth"))

<All keys matched successfully>

In [ ]:
learn_results = model.learn(total_timesteps=5050000, progress_bar=True)